In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tools.agma_equations as agma_equations
import materials.ferrium_c61 as ferrium_c61

In [2]:
velocity_ratio = np.sqrt(0.075)     # Velocity ratio for a single stage
gear_ratio = 1 / velocity_ratio     # Torque ratio for a single stage
load_factor = 1.05                  # Load factor for each branch of the gearbox
safety_factor = 1.5                 # Safety factor for the aerospace industry

# Material propreties for Ferrium C61
allowable_bending_stress = ferrium_c61.bending_fatigue_strength / safety_factor
allowable_surface_stress = ferrium_c61.surface_fatigue_strength / safety_factor ** 0.5 

Np = np.arange(21,38)               # The range of possible number of pinio teeth 
Ng = np.round(Np / velocity_ratio)  # The range of possible gear number of teeth

fw_res = 201                            # Grid resolution for discretizing the face widths
fw_1 = np.linspace(0.8, 5, fw_res)      # Range of face widths for first-stage 
fw_2 = np.linspace(0.8, 5, fw_res)      # Range of face widths for second-stage


In [3]:
# Torque and rotational speed on the first pinion: data found in Table 2 of report
torque_p1 = 3639.71439      # lb-in, Maximum possible torque
rot_speed_p1 = 2094.39510   # rad/s, Maximum possible rotational speed

# Maximum torque and rotational speed on the first gear
# The following equations are derived in the section Torques and Load on Gears 
torque_g1 = torque_p1 / 2 * gear_ratio * load_factor
rot_speed_g1 = rot_speed_p1 * velocity_ratio

# Maximum torque and rotational speed on the second pinion
# The following equations are derived in the section Torques and Load on Gears 
torque_p2 = torque_g1
rot_speed_p2 = rot_speed_g1

# Maximum torque and rotational speed on the second gear
# The following equations are derived in the section Torques and Load on Gears 
torque_g2 = 2 * torque_p2 * gear_ratio * load_factor
rot_speed_g2 = rot_speed_p2 * velocity_ratio

# Uses the function bending_stress defined in the folder tools in the file agma_equations.py
# to calculate the bending and surface stresses on all the gears and pinion on the complete domain
# As shown in the section Optimization Strategy of the report this corresponds to 1,162,851 states 
bending_stresses_p1 = agma_equations.bending_stress(Np, Ng, fw_1, torque_p1, rot_speed_p1, 
                                                    diametral_pitch=10, pinion=True, idler=True)
surface_stresses_p1 = agma_equations.surface_stress(Np, Ng, fw_1, torque_p1, rot_speed_p1, 
                                                    diametral_pitch=10, pinion=True, idler=True)

bending_stresses_g1 = agma_equations.bending_stress(Np, Ng, fw_1, torque_g1, rot_speed_g1, 
                                                    diametral_pitch=10)
surface_stresses_g1 = agma_equations.surface_stress(Np, Ng, fw_1, torque_g1, rot_speed_g1, 
                                                    diametral_pitch=10)

bending_stresses_p2 = agma_equations.bending_stress(Np, Ng, fw_2, torque_p2, rot_speed_p2, 
                                                    diametral_pitch=10, pinion=True)
surface_stresses_p2 = agma_equations.surface_stress(Np, Ng, fw_2, torque_p2, rot_speed_p2, 
                                                    diametral_pitch=10, pinion=True)

bending_stresses_g2 = agma_equations.bending_stress(Np, Ng, fw_2, torque_g2, rot_speed_g2, 
                                                    diametral_pitch=10, idler=True)
surface_stresses_g2 = agma_equations.surface_stress(Np, Ng, fw_2, torque_g2, rot_speed_g2, 
                                                    diametral_pitch=10, idler=True)

In [4]:
# The following code simply creates a mulit-dimensional array with a value of true 
# in indices where the constraint of the the optimization problem are satisfied and a 
# value of false in the indices where the connstraints are not met. The boolean arrays 
# are derived for each gear and pinion individually and then assembled to form a global
# array.

pinion1_design_space = np.logical_and(bending_stresses_p1 < allowable_bending_stress, 
                                     surface_stresses_p1 < allowable_surface_stress)

gear1_design_space = np.logical_and(bending_stresses_g1 < allowable_bending_stress, 
                                     surface_stresses_g1 < allowable_surface_stress)

pinion2_design_space = np.logical_and(bending_stresses_p2 < allowable_bending_stress, 
                                     surface_stresses_p2 < allowable_surface_stress)

gear2_design_space = np.logical_and(bending_stresses_g2 < allowable_bending_stress, 
                                     surface_stresses_g2 < allowable_surface_stress)

stage1_design_space = np.logical_and(gear1_design_space, pinion1_design_space)
stage2_design_space = np.logical_and(gear2_design_space, pinion2_design_space)
stage1_space_aug = np.repeat(stage1_design_space[:,:, np.newaxis], fw_res, axis = 2)
stage2_space_aug = np.repeat(stage2_design_space[:, np.newaxis, :], fw_res, axis = 1)

global_design_space = np.logical_and(stage1_space_aug, stage2_space_aug)

In [5]:
# The total volume for each grid in the domain is computed. This is the objective function
# that was defined in the report in the section Objective Function of the Design Problem 
total_volume = agma_equations.total_gear_volume(Np, Ng, fw_1, fw_2, diametral_pitch=10)
# The value of infinity is assigned to the grid points where the constraints are not 
# satisfied to exlude them from being selected as a minimum
total_volume[np.logical_not(global_design_space)] = np.inf
# The indices of the minimum solution that satisfies the constraints are found
idx1, idx2, idx3 = np.unravel_index(np.argmin(total_volume), total_volume.shape)

print(f"Total volume: {np.round(total_volume[idx1, idx2, idx3], 2)}")
print(f"Number of pinion teeth: {Np[idx1]}\nNumber of gear teeth: {Ng[idx1]}")
print(f"Stage 1 face width: {fw_1[idx2]}\nStage 2 face width: {fw_2[idx3]}")

Total volume: 1172.51
Number of pinion teeth: 37
Number of gear teeth: 135.0
Stage 1 face width: 1.304
Stage 2 face width: 4.769
